In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, Dataset

import spacy
import numpy as np
import random
import math
import time
from tqdm import tqdm
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

In [13]:
torch.backends.cudnn.deterministic = True

In [14]:
#load in the spacy tokenizers
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [15]:
# function to load a document into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [16]:
# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

In [17]:
# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# normalize unicode characters. needed to convert some german characters to ascii
			line = normalize('NFD', line).encode('ascii', 'ignore')
			line = line.decode('UTF-8')
			# tokenize on white space
			line = line.split()
			# remove punctuation from each token
			line = [word.translate(table) for word in line]
			# remove non-printable chars form each token
			line = [re_print.sub('', w) for w in line]
			# remove tokens with numbers in them
			line = [word for word in line if word.isalpha()]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

In [18]:
# load dataset
filename = '../data/deu.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
#use the first 18k sentences to train, validate and test the model
clean_pairs = clean_pairs[0:18000]

In [19]:
np.random.shuffle(clean_pairs)
train_clean = clean_pairs[0:16000]
val_clean = clean_pairs[16000:17000]
test_clean = clean_pairs[17000:]
print("Validation data shape",val_clean.shape)
print("Validation data sentences",val_clean[0:10])
print("Training data shape",train_clean.shape)
print("Validation data sentences",train_clean[0:10])
print("Test data shape",test_clean.shape)
print("Test data sentences",test_clean[0:10])


(1000, 2)
[['You could lead' 'Sie konnten die Fuhrung ubernehmen']
 ['Im in danger' 'Ich bin in Gefahr']
 ['The partys over' 'Die Fete ist beendet']
 ['Can you forgive me' 'Kannst du mir verzeihen']
 ['Hey I know you' 'He dich kenne ich doch']
 ['We know this' 'Wir wissen das']
 ['What was he up to' 'Was hatte er vor']
 ['Nothing happened' 'Nix passiert']
 ['Im not certain' 'Ich bin mir nicht sicher']
 ['He is very tall' 'Er ist sehr gro']]
(16000, 2)
[['I have a big house' 'Ich habe ein groes Haus']
 ['Lets get off here' 'Lasst uns hier aussteigen']
 ['Thats so annoying' 'Das ist so nervig']
 ['Tom was impolite' 'Tom war unhoflich']
 ['The book is white' 'Das Buch ist wei']
 ['Look what I did' 'Schau mal was ich gemacht habe']
 ['Give Tom a kiss' 'Gib dem Tom einen Kuss']
 ['Dont let me down' 'Lassen Sie mich nicht im Stich']
 ['It may seem odd' 'Es mag eigenartig erscheinen']
 ['Youre being mean' 'Du bist gemein']]
(1000, 2)
[['Did Tom need help' 'Hat Tom Hilfe gebraucht']
 ['Tom bou

In [20]:
#I stored the english senetence from the pair to train.en and german senetence to train.de.
#Similarly for the validation and test data
#I did this because I couldn't figure out how to use a dataset in torchtext to load the sentences. So I used to Multi30k structure.
#Something to work on in the future.
eng_file = "./.data/multi30k/train.en"
ge_file = "./.data/multi30k/train.de"
en_pt = open(eng_file,"w")
ge_pt = open(ge_file,"w")

for p in train_clean:
    en_pt.write(p[0]+"\n")
    ge_pt.write(p[1]+"\n")
    
en_pt.close()
ge_pt.close()

eng_file = "./.data/multi30k/val.en"
ge_file = "./.data/multi30k/val.de"
en_pt = open(eng_file,"w")
ge_pt = open(ge_file,"w")

for p in val_clean:
    en_pt.write(p[0]+"\n")
    ge_pt.write(p[1]+"\n")
    
en_pt.close()
ge_pt.close()

eng_file = "./.data/multi30k/test2016.en"
ge_file = "./.data/multi30k/test2016.de"
en_pt = open(eng_file,"w")
ge_pt = open(ge_file,"w")

for p in test_clean:
    en_pt.write(p[0]+"\n")
    ge_pt.write(p[1]+"\n")
    
en_pt.close()
ge_pt.close()

In [21]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [22]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [23]:
#lower argument converts all strings to lower case
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

C:\Users\MOHIT\Anaconda3\envs\torch\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [24]:
#load in training, validation and test data
train_data,valid_data,test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

C:\Users\MOHIT\Anaconda3\envs\torch\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [25]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 16000
Number of validation examples: 1000
Number of testing examples: 1000


In [26]:
print(vars(train_data.examples[39]))

{'src': ['lassen', 'wir', 'einen', 'drachen', 'steigen'], 'trg': ['lets', 'fly', 'a', 'kite']}


In [27]:
#build the vocabulary with a minimum threshold of 2 for word count
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [28]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 2928
Unique tokens in target (en) vocabulary: 2175


In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

C:\Users\MOHIT\Anaconda3\envs\torch\lib\site-packages\torchtext\data\iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [31]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))

        #embedded = [sentence len, batch size, emb dim]
        #outputs (the top-layer hidden state for each time-step), hidden (the final hidden state for each layer, 
        #h_T, stacked on top of each other) and cell (the final cell state for each layer, c_T, 
        #stacked on top of each other).
        #n_directions=1 because the LSTM is not bidirectional
        
        outputs, (hidden, cell) = self.rnn(embedded)
        #outputs = [sentence len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        
        return hidden, cell

In [32]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim,)
        
        self.dropout = nn.Dropout(dropout)
        
        #self.softmax = nn.LogSoftmax(dim=0)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        #output of cell state and hidden state of encoder go to the decoder
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        #print(prediction.shape)
        #prediction = self.softmax(prediction)
        #print(prediction.shape)
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [33]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            #print(top1.shape)
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [46]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [47]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2928, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(2175, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=2175, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [48]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,778,559 trainable parameters


In [49]:
optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

In [50]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX) 

In [51]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(tqdm(iterator)):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        #applying gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        

    return epoch_loss / len(iterator)

In [52]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(tqdm(iterator)):

            src = batch.src
            #print("src1",src.shape)
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            #print("output b4",output.shape)
            #print(output[0,0,0])
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [53]:
def evaluate_sentence(sentence):
    sentence = sentence.lower()
    
    #tokenize german senetece
    tokens = tokenize_de(sentence)
    #insert sos token at start of sentence
    tokens.insert(0, SRC.init_token)
    tokens.append(SRC.eos_token)
    
    #convert words to numbers according to the vobaulary
    text_to_indices = [SRC.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)
    
    #output begins with sos, so append it to the output
    outputs = [TRG.vocab.stoi["<sos>"]]
    
    #while predicting, the previous output is the input to the next step
    for _ in range(50):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)
        
        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == TRG.vocab.stoi["<eos>"]:
            break
    
    #convert the numbers to words using vocabulary
    translated_sentence = [TRG.vocab.itos[idx] for idx in outputs]
    print(translated_sentence[1:])
    

In [54]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [67]:
model.load_state_dict(torch.load('../models/model-v4.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 48.18it/s]

| Test Loss: 2.061 | Test PPL:   7.854 |


In [55]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    scheduler.step(valid_loss)
    evaluate_sentence("Wir haben eine gefunden")
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '../models/model-v4.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 111.41it/s]


['we', 're', 'a', '<eos>']


  1%|▉                                                                                 | 3/250 [00:00<00:08, 27.60it/s]

Epoch: 01 | Time: 0m 9s
	Train Loss: 4.296 | Train PPL:  73.408
	 Val. Loss: 3.760 |  Val. PPL:  42.951


  0%|                                                                                          | 0/250 [00:00<?, ?it/s]

...

['we', 'found', 'one', '<eos>']
Epoch: 66 | Time: 0m 27s
	Train Loss: 0.146 | Train PPL:   1.158
	 Val. Loss: 2.187 |  Val. PPL:   8.913


  0%|▎                                                                                 | 1/250 [00:00<00:38,  6.49it/s]

['we', 'found', 'one', '<eos>']
Epoch: 67 | Time: 0m 27s
	Train Loss: 0.148 | Train PPL:   1.160
	 Val. Loss: 2.188 |  Val. PPL:   8.920


 64%|██████████████████████████████████████████████████▉                             | 159/250 [00:17<00:10,  9.10it/s]


KeyboardInterrupt: 

In [57]:
model.load_state_dict(torch.load('../models/model-v4.pt'))

test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

#random.choice(test_clean)
test_file = open("./.data/multi30k/test2016.de")
test_sentences = []
for sentence in test_file:
    sentence = sentence.strip("\n")
    test_sentences.append(sentence)
    
test_file.close()
test_file = open("./.data/multi30k/test2016.en")
ground_truth = []
for sentence in test_file:
    sentence = sentence.strip("\n")
    ground_truth.append(sentence)
test_file.close()
#print(test_sentences[0:10])
i=0
while i<10:
    pair_choice = random.randint(0, 1000)
    print(pair_choice)
    ground_truth_sen = ground_truth[pair_choice]
    ground_truth_sen = ground_truth_sen.lower()
    to_test = test_sentences[pair_choice]
    print("GT",ground_truth_sen)
    print("TT",to_test)
    evaluate_sentence(to_test.lower())
    #print(ground_truth)
    i+=1

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 26.34it/s]


| Test Loss: 2.022 | Test PPL:   7.551 |
478
GT what a surprise
TT Was fur eine Uberraschung
['what', 'a', 'shame', '<eos>']
710
GT i looked for a job
TT Ich suchte nach einer Arbeit
['i', 'slapped', 'a', 'day', '<eos>']
51
GT hes a man now
TT Er ist jetzt ein Mann
['he', 's', 'already', 'a', 'age', '<eos>']
533
GT tom whistled
TT Tom pfiff
['tom', 'is', '<eos>']
428
GT is tom married
TT Ist Tom verheiratet
['is', 'tom', 'married', '<eos>']
957
GT the answer is
TT Die Antwort lautet zweiundvierzig
['the', '<unk>', 'is', 'melting', '<eos>']
957
GT the answer is
TT Die Antwort lautet zweiundvierzig
['the', '<unk>', 'is', 'melting', '<eos>']
101
GT pass me the salt
TT Gib mir bitte das Salz ruber
['please', 'give', 'me', 'a', '<eos>']
672
GT youre in trouble
TT Du steckst in Schwierigkeiten
['you', 're', 'in', 'trouble', '<eos>']
45
GT this book is small
TT Das Buch ist dunn
['the', 'book', 'is', 'cold', '<eos>']
